In [1]:
import os
import sys
import csv
import numpy as np
import gurobipy as gp
import matplotlib.pyplot as plt
from pathlib import Path
from scipy.io import loadmat
from time import perf_counter as timer

from src import *

In [2]:
thermal = Thermal()
renewable = Renewable()
demand = Demand()
commitment = Commitment()
data = Data()

data.load_thermal(thermal)
data.load_renewable_capacity(renewable)
data.load_renewable_generation(renewable)
data.load_demand(demand)
data.load_commitment_decision(commitment)

In [3]:
class Results:
    def __init__(self):
        self.objval = []
        self.smp = []
        self.slack = []
        self.offunit_bool = []
        self.runtime = []
        self.total_runtime = []

    def collect(self, objval, smp, slack, offunit_bool, runtime, total_runtime):
        self.objval.append(objval)
        self.smp.append(smp)
        self.slack.append(slack)
        self.offunit_bool.append(offunit_bool)
        self.runtime.append(runtime)
        self.total_runtime.append(total_runtime)


    def convert_list_to_nparray(self):
        self.objval = np.array(self.objval)
        self.smp = np.array(self.smp)
        self.slack = np.array(self.slack)
        self.offunit_bool = np.array(self.offunit_bool)
        self.runtime = np.array(self.runtime)
        self.total_runtime = np.array(self.total_runtime)

In [4]:
results = Results()

In [5]:
renewable.solar_generation.shape

(8760, 197)

In [11]:
idx_hour = 0

model = gp.Model()

hourly_decision = commitment.decision[idx_hour]


p_thermal = model.addVars(
    thermal.count,
    lb=(thermal.pmin * hourly_decision).tolist(),
    ub=(thermal.pmax * hourly_decision).tolist(),
)

p_solar = model.addVars(renewable.count, lb=0, ub=renewable.solar_generation[idx_hour].tolist())
p_wind = model.addVars(renewable.count, lb=0, ub=renewable.wind_generation[idx_hour].tolist())
p_hydro = model.addVars(renewable.count, lb=0, ub=renewable.hydro_generation[idx_hour].tolist())

sum_p_thermal =  gp.quicksum(p_thermal[g] for g in range(thermal.count))
sum_p_solar = gp.quicksum(p_solar[b] for b in range(renewable.count))
sum_p_wind = gp.quicksum(p_wind[b] for b in range(renewable.count))
sum_p_hydro = gp.quicksum(p_hydro[b] for b in range(renewable.count))

model.addConstr(sum_p_thermal + sum_p_solar + sum_p_wind + sum_p_hydro == demand.total[idx_hour])

total_cost_thermal = gp.quicksum(
    thermal.c2.tolist()[g] * p_thermal[g] * p_thermal[g] + thermal.c1.tolist()[g] * p_thermal[g] + (thermal.c0.tolist() * hourly_decision).tolist()[g]
    for g in range(thermal.count)
)

model.setObjective(total_cost_thermal, gp.GRB.MINIMIZE)

In [12]:
model.setParam("OutputFlag", 0)
model.optimize()

GurobiError: Model too large for size-limited license; visit https://gurobi.com/unrestricted for more information

In [ ]:
for idx_hour in range(8760):
    timer_start = timer()
    model = gp.Model()

    p = model.addVars(
        thermal.count,
        lb=(thermal.pmin * commitment.decision[idx_hour]).tolist(),
        ub=(thermal.pmax * commitment.decision[idx_hour]).tolist(),
    )

    model.addConstr(
        gp.quicksum(p[g] for g in range(thermal.count)) == 
        demand.total[idx_hour] - renewable.total_generation[idx_hour],
    )

    model.setObjective(
        gp.quicksum(
            thermal.c2.tolist()[g] * p[g] * p[g] + thermal.c1.tolist()[g] * p[g] + (thermal.c0 * commitment.decision[idx_hour]).tolist()[g]
            for g in range(thermal.count)
        ),
        gp.GRB.MINIMIZE
    )

    model.setParam("OutputFlag", 0)
    model.optimize()
    timer_end = timer()

    if model.Status == gp.GRB.OPTIMAL:
        objval = model.ObjVal
        smp = model.getAttr("Pi")[0]
        slack = model.getAttr("Slack")[0]
        offunit_bool = np.all(np.array(model.getAttr("X"))[np.where(1 - commitment.decision[idx_hour])[0]] == 0)
        runtime = model.Runtime
        total_runtime = timer_end - timer_start

        results.collect(objval, smp, slack, offunit_bool, runtime, total_runtime)

In [ ]:
results.convert_list_to_nparray()

In [ ]:
np.all(results.offunit_bool), results.slack.sum()

In [ ]:
results.runtime.mean(), (results.total_runtime - results.runtime).mean(), results.total_runtime.mean()

In [ ]:
for idx_hour in range(8760):
    timer_start = timer()
    model = gp.Model()

    p = model.addVars(
        thermal.count,
        lb=(thermal.pmin * commitment.decision[idx_hour]).tolist(),
        ub=(thermal.pmax * commitment.decision[idx_hour]).tolist(),
    )

    model.addConstr(
        gp.quicksum(p[g] for g in range(thermal.count)) == 
        demand.total[idx_hour] - renewable.total_generation[idx_hour],
    )

    model.setObjective(
        gp.quicksum(
            thermal.c2[g].tolist() * p[g] * p[g] + thermal.c1[g].tolist() * p[g]
            for g in range(thermal.count)
        ),
        gp.GRB.MINIMIZE
    )

    model.setParam("OutputFlag", 0)
    model.optimize()
    timer_end = timer()

    if model.Status == gp.GRB.OPTIMAL:
        objval = model.ObjVal
        smp = model.getAttr("Pi")[0]
        slack = model.getAttr("Slack")[0]
        offunit_bool = np.all(np.array(model.getAttr("X"))[np.where(1 - commitment.decision[idx_hour])[0]] == 0)
        runtime = model.Runtime
        total_runtime = timer_end - timer_start

        results.collect(objval, smp, slack, offunit_bool, runtime, total_runtime)

    # elif model.Status == gp.GRB.INFEASIBLE:
    #     model.computeIIS()
    #     checker = 

In [ ]:
    # if model.Status == gp.GRB.OPTIMAL:
    #     result_smp.append(model.getAttr("Pi", model.getConstrs())[0])
    #     result_noncommitp0true.append(np.all(np.array(model.getAttr("X", p.values()))[np.where(commitment.decision[idx_hour] == 0)[0]] == 0))
    #     result_slack.append(model.getAttr("Slack", model.getConstrs())[0])
    #     result_runtime.append(model.Runtime)

    # elif model.Status == gp.GRB.INFEASIBLE:
    #     model.computeIIS()
    #     iis_char = "n"
    #     yeah = model.getVars()[0]
    #     if yeah.IISUB:
    #         iis_char = "u"
    #     elif yeah.IISLB:
    #         iis_char = "l"
    #     result_infeasible.append(iis_char)

    # else:
    #     raise ValueError("wtf")